In [1]:
import os

In [2]:
%pwd

'c:\\Users\\madhu\\deep learning\\text_summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
pwd

'c:\\Users\\madhu\\deep learning\\text_summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)

class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path
    

In [6]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class configurationmanager:
    def __init__(
            self,
            config_filepath= CONFIG_FILE_PATH,
            params_filepath= PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir)
        
        return data_ingestion_config

In [8]:
import os
import requests
import urllib.request as request
import zipfile
from src.textSummarizer.logging import logger
from src.textSummarizer.utils.common import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            response = requests.get(self.config.source_url, stream=True)
            response.raise_for_status()  # Check if the request was successful

            with open(self.config.local_data_file, 'wb') as file:
                for chunk in response.iter_content(chunk_size=8192):
                    file.write(chunk)
            logger.info(f"{self.config.local_data_file} download: {response.headers}")
        else:
            logger.info(f"File already exits of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        Zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Extracted files from {self.config.local_data_file} to {unzip_path}")


In [9]:
try: 
    config = configurationmanager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-12 11:04:40,798: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-12 11:04:40,804: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-12 11:04:40,807: INFO: common: created directory at :artifacts]
[2025-03-12 11:04:40,809: INFO: common: created directory at :artifacts/data_ingestion]


[2025-03-12 11:04:44,404: INFO: 2114711410: artifacts/data_ingestion/archive.zip download: {'Connection': 'keep-alive', 'Content-Length': '8377572', 'Cache-Control': 'max-age=300', 'Content-Security-Policy': "default-src 'none'; style-src 'unsafe-inline'; sandbox", 'Content-Type': 'application/zip', 'ETag': 'W/"842c892e354d78f841533d9b67ebf1cf6bb390c47eb0a005d13e8f167901732b"', 'Strict-Transport-Security': 'max-age=31536000', 'X-Content-Type-Options': 'nosniff', 'X-Frame-Options': 'deny', 'X-XSS-Protection': '1; mode=block', 'X-GitHub-Request-Id': '42D3:50816:BB9C4:2156E1:67D11C66', 'Accept-Ranges': 'bytes', 'Date': 'Wed, 12 Mar 2025 05:34:41 GMT', 'Via': '1.1 varnish', 'X-Served-By': 'cache-ccu830029-CCU', 'X-Cache': 'HIT', 'X-Cache-Hits': '0', 'X-Timer': 'S1741757682.716592,VS0,VE1', 'Vary': 'Authorization,Accept-Encoding,Origin', 'Access-Control-Allow-Origin': '*', 'Cross-Origin-Resource-Policy': 'cross-origin', 'X-Fastly-Request-ID': 'dc35924582d0984b7c280b8dec275435239083b4', 'Exp